In [4]:
# !pip install google-search-results
import pandas as pd
# api_key = "e4fb6e74a565645df616278e366b117d5d63cc04d23182ce7fca8c25ec2fda1b"
api = 'apify_api_BqCUn7b2esAHLm3kQPVRagWEykcmMT3u8Gv7'

In [ ]:
from serpapi import GoogleSearch
import json

In [ ]:
search_term = "Data Science"
search_location = "India"
params = {
    "api_key" : api_key,
    "device" : "desktop",
    "engine" : "google_jobs",
    "q" : search_term,
    "hl": 'en',
    'gl': 'in',
    "start" : 0 #pagination
}

search = GoogleSearch(params)

In [ ]:
data = search.get_dict()

In [ ]:
data.keys()

In [ ]:
data['search_metadata']

In [ ]:
with open(r"C:\Users\Jadha\PycharmProjects\python-stuff\sample.json", "w") as outfile:
    json.dump(data['jobs_results'], outfile)

In [ ]:
len(data['jobs_results'])

In [ ]:
data['jobs_results']

In [ ]:
job = data['jobs_results'][0]

In [ ]:
print(job.keys())

In [ ]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])
# job['job_id']

In [ ]:
job = data['jobs_results'][4]

In [ ]:
job

In [ ]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])

## Get Combine Json data for 5 pages for dataScience

In [ ]:
metadata = []
results = []

for i in range(5):
    search = GoogleSearch({**params, 'start' : i*10})
    page_data = search.get_dict()
    metadata.append(page_data['search_metadata'])
    results += page_data['jobs_results']

## Create CSV file
extra features can be added like `employee website`

`Key Points` - Some day ago | fulltime or PartTime | worktype

In [ ]:
data = {
    'title': [],
    'company_name': [],
    'location': [],
    'from': [],
    'Benefits': [],
    'Qualifications': [],
    'Responsibilities': [],
}

highlight_keys = ['Benefits', 'Qualifications', 'Responsibilities']

for job in results:
    data['title'].append(job['title'])
    data['company_name'].append(job['company_name'])
    data['location'].append(job['location'])
    data['from'].append(job['via'])
    
    for k in highlight_keys:
        found = False
        for h in job['job_highlights']:
            if 'title' in h.keys() and k == h['title']:
                data[k].append(". ".join(h['items']))
                found = True
        if not found:
            data[k].append("")


In [ ]:
df = pd.DataFrame.from_dict(data)
df.to_csv('main.csv', index=False)

In [ ]:
df

# Apify API

In [5]:
from apify_client import ApifyClient
# Initialize the ApifyClient with your API token
client = ApifyClient(token=api)

In [100]:
# Prepare the actor input
run_input = {
    "queries": "Microsoft fresher jobs in India",
    "maxPagesPerQuery": 100,
    "countryCode": "in",
    "languageCode" : "en",
    "locationUule" : r"w+CAIQICIFU3VyYXQ=",
    "radius" : 500,
}

# Run the actor and wait for it to finish
run = client.actor("dan.scraper/google-jobs-scraper").call(run_input=run_input)

# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)


{'searchQuery': {'term': 'Microsoft fresher jobs in India', 'page': 1, 'type': 'SEARCH', 'domain': 'google.co.in', 'countryCode': 'IN', 'languageCode': 'en', 'locationUule': 'w CAIQICIFU3VyYXQ=', 'resultsPerPage': 10}, 'url': 'https://www.google.co.in/search?hl=en&ibp=htl%3Bjobs&lrad=500&q=Microsoft%20fresher%20jobs%20in%20India&uule=w+CAIQICIFU3VyYXQ=', 'hasNextPage': True, 'googleJobs': [{'title': 'Microsoft is hiring Software Engineer [2022 Grad] (21 February 2022)', 'companyName': 'Microsoft', 'location': 'India', 'via': 'via Workat.tech', 'description': 'Microsoft is hiring Software Engineer [2022 Grad] (21 February 2022). Apply now at https://workat.tech/company/microsoft/jobs/software-engineer-2022-grad-Q8FvRWMg', 'relatedLinks': [{'link': 'http://www.microsoft.com/', 'text': 'microsoft.com'}, {'link': 'https://www.google.com/search?hl=en&q=Microsoft&sa=X&ved=0ahUKEwiX9euW3KD9AhUZpokEHWi3BBwQmJACCNsK', 'text': 'See web results for Microsoft'}], 'thumbnail': 'https://encrypted-tb

In [101]:
# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item.keys())
    print(len(item['googleJobs']))

dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs'])
0


In [102]:
data = {
    'title': [],
    'company_name': [],
    'location': [],
    'from': [],
    'Benefits': [],
    'Qualifications': [],
    'Responsibilities': [],
    'thumbnail': [],
    'applyLink': [],
}

highlight_keys = ['Benefits', 'Qualifications', 'Responsibilities']

for page in client.dataset(run["defaultDatasetId"]).iterate_items():
    page_data = page['googleJobs']
    for job in page_data:
#         if 'jobHighlights' not in job.keys():
#             continue
        data['title'].append(job['title'])
        data['company_name'].append(job['companyName'])
        data['location'].append(job['location'] if 'location' in job.keys() else "") 
        data['from'].append(job['via'])
#         print(job['applyLink']['link'])
        data['applyLink'].append(job['applyLink']['link'])
        data['thumbnail'].append(job['thumbnail'] if 'thumbnail' in job.keys() else "")
        for k in highlight_keys:
            if 'jobHighlights' in job.keys():
                found = False
                for h in job['jobHighlights']:
                    if 'title' in h.keys() and k == h['title']:
                        data[k].append(". ".join(h['items']))
                        found = True
                if not found:
                    data[k].append("")
            else:
                data[k].append("")


In [103]:
df = pd.DataFrame.from_dict(data)
print(len(df))
display(df.head())

20


,title,company_name,location,from,Benefits,Qualifications,Responsibilities,thumbnail,applyLink
0,Microsoft is hiring Software Engineer [2022 Gr...,Microsoft,India,via Workat.tech,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://workat.tech/company/microsoft/jobs/sof...
1,Microsoft D365-Functional-Finance,DXC Technology,"Gurugram, Haryana, India",via LinkedIn,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://in.linkedin.com/jobs/view/microsoft-d3...
2,Microsoft Off Campus Drive 2023 Hiring Data An...,Microsoft,"Hyderabad, Telangana, India",via Freshershunt,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://freshershunt.in/microsoft-off-campus-d...
3,Microsoft is hiring for Freshers as Research I...,Microsoft,"Bengaluru, Karnataka, India",via DJobBuzz.com,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.djobbuzz.com/home/viewjob/7447/mic...
4,Microsoft Jobs 2023 for Research Fellow,Microsoft,"Bengaluru, Karnataka, India",via Off Campus Careers,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://offcampuscareers.com/microsoft-jobs-4/...


In [92]:
df['company_name'].nunique()

20

In [104]:
df.to_csv("data_science_job_data_with_applylink_micro.csv", index=False)